In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
bikers_df = pd.read_csv("/kaggle/input/bicycle-accidents-in-great-britain-1979-to-2018/Bikers.csv")
accidents_df = pd.read_csv("/kaggle/input/bicycle-accidents-in-great-britain-1979-to-2018/Accidents.csv")

In [ ]:
combined_df = bikers_df.merge(accidents_df, on='Accident_Index', how='inner')
combined_df

,Accident_Index,Gender,Severity,Age_Grp,Number_of_Vehicles,Number_of_Casualties,Date,Time,Speed_limit,Road_conditions,Weather_conditions,Day,Road_type,Light_conditions
0,197901A1SEE71,Male,Serious,36 to 45,2,1,1979-01-01,18:20,50.0,Snow,Unknown,Monday,Dual carriageway,Darkness lights lit
1,197901A2JDW40,Male,Slight,46 to 55,1,1,1979-02-01,09:15,30.0,Snow,Unknown,Tuesday,Unknown,Daylight
2,197901A4IJV90,Male,Slight,46 to 55,2,1,1979-04-01,08:45,30.0,Snow,Unknown,Thursday,Unknown,Daylight
3,197901A4NIE33,Male,Slight,36 to 45,2,1,1979-04-01,13:40,30.0,Wet,Unknown,Thursday,Unknown,Daylight
4,197901A4SKO47,Male,Slight,46 to 55,2,1,1979-04-01,18:50,30.0,Wet,Unknown,Thursday,Unknown,Darkness lights lit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827856,2018983118818,Male,Slight,6 to 10,2,1,2018-02-07,14:55,30.0,Dry,Clear,Monday,Single carriageway,Daylight
827857,2018983119218,Male,Serious,56 to 65,2,1,2018-07-24,07:45,30.0,Dry,Clear,Tuesday,Single carriageway,Daylight
827858,2018983120618,Male,Serious,11 to 15,2,1,2018-10-08,13:25,20.0,Dry,Clear,Friday,Single carriageway,Daylight
827859,2018983121918,Male,Serious,46 to 55,2,1,2018-07-18,21:10,30.0,Dry,Clear,Wednesday,Single carriageway,Daylight


In [ ]:
null_values = combined_df.isnull()
null_counts = null_values.sum()
print(null_counts[null_counts > 0])

Series([], dtype: int64)


In [ ]:
combined_df.size

11590054

In [ ]:
features = ['Number_of_Vehicles', 'Speed_limit', 'Road_conditions', 'Weather_conditions', 'Road_type', 'Light_conditions','Severity','Time']
target = 'Accident_Index'

X = combined_df[features]
y = combined_df[target]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
encoder = TargetEncoder()

In [ ]:
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)

In [ ]:
scaler = StandardScaler()
X_train_encoded = scaler.fit_transform(X_train_encoded)
X_test_encoded = scaler.transform(X_test_encoded)

In [ ]:
# Build and train the neural network
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_dim=X_train_encoded.shape[1]),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Predict on the test data
y_pred = model.predict(X_test_encoded)

# Convert predictions to binary (e.g., 0 or 1) depending on your problem
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")
